In [6]:
import pandas as pd

import warnings
warnings.simplefilter("ignore")

In [1]:
#Define a function for processing data from POLON
def polon_data(path_polon):
    
    #Get data from Excel file
    polon = pd.read_excel(path_polon)

    #Selecting the necessary range from the data. Academic teachers from the selected university
    polon = polon[polon['ZATRUDNIENIE'].str.contains('Politechnika Warszawska') &
                       polon['WIELKOSC_ETATU_PREZENTACJA_DZIESIETNA']!=0]

    #Due to the lack of flexibility in entering data into POLON, 
    #some positions are entered as "other", so we use the names from the column next to it
    polon.loc[polon['STANOWISKO_NAZWA'] == 'Inne', 'STANOWISKO_NAZWA'] = polon['STANOWISKO_INNE']

    #Combine name and last name in one. Change it to capital letter
    polon['NAZWISKO_IMIE'] = polon['NAZWISKO']+' '+ polon['IMIE']
    polon['NAZWISKO_IMIE'] = polon['NAZWISKO_IMIE'].str.capitalize()

    #Change orders of the columns
    polon = polon[['ZATRUDNIENIE', 'PESEL_NR_DOK_TOZSAM', 'NAZWISKO_IMIE', 'GRUPA_STANOWISK', 
                   'STANOWISKO_NAZWA', 'WIELKOSC_ETATU_PREZENTACJA_DZIESIETNA', 'PLEC', 
                   'STOPIEN_TYTUL_AKTUALNY_NA_DZIEN_WYGENEROWANIA_RAPORTU',
                   'PODSTAWOWE_MIEJSCE_PRACY', 'OBYWATELSTWO']]

    #Change headers to capital letter
    polon.columns = polon.columns.str.capitalize()

    #Change names of columns, to more readable and unified
    polon = polon.rename(columns={'Pesel_nr_dok_tozsam':'PESEL',
                                  'Wielkosc_etatu_prezentacja_dziesietna':'Etat',
                                  'Stopien_tytul_aktualny_na_dzien_wygenerowania_raportu':'Tytuł',
                                  'Podstawowe_miejsce_pracy':'Podst. miej. pracy',
                                  'Stanowisko_nazwa':'Stanowisko',
                                  'Plec':'Płeć'})

    #Unified formatting. Change on int64 is necessery, its key value to merge data.
    polon['PESEL'] = polon['PESEL'].astype('int64')
    
    #Unification of full-time positions
    polon['Etat'] = polon['Etat'].replace(',','.',regex=True)
    polon['Etat'] = polon['Etat'].astype('float64')

    #Change of record on that same as in SAP data
    polon['Stanowisko'] = polon['Stanowisko'].str.lower()

    
    #Change of record on that same as in SAP data, using a dictionery
    dict_grupa_stanowisk = {'Dyplomowany bibliotekarz oraz pracownik dokumentacji i informacji naukowej' : 'Dydaktyczni',
                            'Pracownik badawczo-dydaktyczny' : 'Badawczo-dydaktyczni',
                            'Pracownik badawczy' : 'Badawczy',
                            'Pracownik dydaktyczny' : 'Dydaktyczni',
                            'Pracownik inżynieryjno-techniczny' : 'Inżynieryjno-techn.',
                            'Pracownik naukowo-techniczny' : 'Naukowo-techniczni',
                            'Pracownik niebędący nauczycielem akademickim' : 'Pozostali nienauczyciele'
                           }
    polon.replace({'Grupa_stanowisk' : dict_grupa_stanowisk}, inplace=True)

    #Change of record on that same as in SAP data, using a dictionery
    dict_plec = {'M' : 'Mężczyzna',
                 'K' : 'Kobieta'
                }
    polon.replace({'Płeć' : dict_plec}, inplace=True)

    #Change of record on that same as in SAP data, using a dictionery
    dict_tytuł = {'dr' : 'doktor',
                            'dr hab.' : 'dr habilitowany',
                            'lic.' : 'licencjat',
                            'mgr' : 'magister',
                            'mgr inż.' : 'mgr inżynier',
                            'mgr, inż.' : 'mgr inżynier',
                            'prof. dr' : 'profesor',
                            'prof. dr hab.' : 'profesor',
                            'inż.':'inżynier'
                            }
    polon.replace({'Tytuł' : dict_tytuł}, inplace=True)

    #Change of record on that same as in SAP data, using a dictionery
    dict_obywatelstwo = {'kazachskie' : 'kazachstańskie'}
    polon.replace({'Obywatelstwo' : dict_obywatelstwo}, inplace=True)

    #Remove duplicates if exist, add prefix to distinguish where the data comes from
    polon = polon.drop_duplicates() 
    polon = polon.add_prefix('POLON_')

    #Prepare two dataframes on based on main groups
    polon_t = polon[polon['POLON_Zatrudnienie'].str.contains('Nauczyciel akademicki')]
    polon_nt = polon[polon['POLON_Zatrudnienie'].str.contains('Nauczyciel akademicki')==False]
    
    return polon_t, polon_nt